In [ ]:
# Import des bibliothèques nécessaires
import sys
import os
sys.path.append('../src')

import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pathlib import Path

# Import de nos modules personnalisés
from data_utils import DataManager
from model_architecture import ImageClassifierBuilder, create_data_augmentation_layer, create_preprocessing_layer

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

# Configuration pour les graphiques
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


In [ ]:
# Initialisation du DataManager
data_manager = DataManager(data_dir="../data")

# Vérification et téléchargement du dataset si nécessaire
if not data_manager.verify_dataset_structure():
    print("Dataset non trouvé. Tentative de téléchargement...")
    success = data_manager.download_dataset()
    if not success:
        print("⚠️ Échec du téléchargement automatique.")
        print("Veuillez télécharger manuellement le dataset depuis:")
        print("https://www.kaggle.com/datasets/anthonytherrien/image-classification-dataset-32-classes")
        print("Et l'extraire dans le dossier '../data/raw/'")
else:
    print("✅ Dataset trouvé et vérifié!")

# Obtenir les informations sur le dataset
dataset_info = data_manager.get_dataset_info()
print(f"\n📊 Informations sur le dataset:")
print(f"Nombre de classes: {dataset_info['num_classes']}")
print(f"Nombre total d'images: {dataset_info['total_images']}")
print(f"Classes: {dataset_info['class_names'][:10]}...")  # Afficher les 10 premières classes


In [ ]:
# Chargement des datasets pour visualisation
if dataset_info.get('num_classes', 0) > 0:
    # Charger un petit échantillon pour la visualisation
    train_ds, val_ds = data_manager.load_datasets(
        image_size=(224, 224),
        batch_size=16,  # Plus petit batch pour la visualisation
        validation_split=0.2
    )
    
    # Visualiser quelques échantillons
    plt.figure(figsize=(15, 10))
    
    # Prendre le premier batch
    for images, labels in train_ds.take(1):
        for i in range(min(12, len(images))):
            plt.subplot(3, 4, i + 1)
            # Convertir l'image en format affichable
            img = images[i].numpy().astype("uint8")
            plt.imshow(img)
            
            # Obtenir le nom de la classe
            class_idx = tf.argmax(labels[i]).numpy()
            class_name = train_ds.class_names[class_idx]
            plt.title(f"Classe: {class_name}", fontsize=10)
            plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(f"📊 Échantillons du dataset visualisés")
    print(f"Classes disponibles: {train_ds.class_names}")
else:
    print("⚠️ Dataset non disponible pour la visualisation")


In [ ]:
# Configuration d'entraînement
training_config = {
    'architecture': 'resnet50',        # Options: resnet50, efficientnet_b0, vgg16, custom_cnn
    'epochs': 20,                      # Nombre d'époques (réduit pour le test)
    'batch_size': 32,                  # Taille du batch
    'learning_rate': 0.001,            # Taux d'apprentissage
    'image_size': (224, 224),          # Taille des images
    'validation_split': 0.2,           # Fraction pour la validation
    'trainable_base': False,           # Transfer learning avec base gelée
    'optimizer': 'adam',               # Optimiseur
    'early_stopping_patience': 5      # Patience pour l'arrêt précoce
}

print("🔧 Configuration d'entraînement:")
for key, value in training_config.items():
    print(f"  {key}: {value}")

print(f"\n💡 Conseils:")
print(f"  - Commencez avec peu d'époques (10-20) pour tester")
print(f"  - Utilisez ResNet50 pour de bonnes performances")
print(f"  - Réduisez batch_size si vous manquez de mémoire GPU")


In [ ]:
# Import du module d'entraînement
from train import ModelTrainer
from datetime import datetime

print("🚀 Préparation de l'entraînement...")

# Créer une instance du trainer avec notre configuration
trainer = ModelTrainer(training_config)

print("✅ Trainer initialisé avec succès!")
print(f"📁 Dossier de sortie: {trainer.output_dir}")
print(f"📊 Dossier des logs: {trainer.log_dir}")

# Afficher un résumé avant de commencer
print(f"\n📋 Résumé de l'entraînement:")
print(f"  Architecture: {training_config['architecture']}")
print(f"  Époques: {training_config['epochs']}")
print(f"  Batch size: {training_config['batch_size']}")
print(f"  Learning rate: {training_config['learning_rate']}")

print(f"\n⏰ Temps estimé: ~{training_config['epochs'] * 2} minutes (selon votre matériel)")
print(f"💾 Les modèles seront sauvegardés automatiquement")
print(f"📈 Suivez les métriques en temps réel avec TensorBoard: tensorboard --logdir=../logs")


In [ ]:
# Étape 1: Préparation des données
print("📥 Étape 1/3: Préparation des données...")

try:
    trainer.prepare_data()
    print("✅ Données préparées avec succès!")
    
    # Afficher les informations sur le dataset
    print(f"\n📊 Informations du dataset:")
    print(f"  Nombre de classes: {trainer.dataset_info['num_classes']}")
    print(f"  Total d'images: {trainer.dataset_info['total_images']}")
    print(f"  Classes: {trainer.dataset_info['class_names'][:5]}... (et {trainer.dataset_info['num_classes']-5} autres)")
    
except Exception as e:
    print(f"❌ Erreur lors de la préparation des données: {e}")
    print("💡 Vérifiez que le dataset est disponible ou téléchargeable depuis Kaggle")


In [ ]:
# Étape 2: Construction du modèle
print("🏗️ Étape 2/3: Construction du modèle...")

try:
    trainer.build_model()
    print("✅ Modèle construit avec succès!")
    
    # Afficher un résumé du modèle
    print(f"\n🧠 Résumé du modèle:")
    trainer.model.summary()
    
    # Compter les paramètres
    total_params = trainer.model.count_params()
    trainable_params = sum([tf.keras.backend.count_params(w) for w in trainer.model.trainable_weights])
    non_trainable_params = total_params - trainable_params
    
    print(f"\n📊 Paramètres du modèle:")
    print(f"  Total: {total_params:,}")
    print(f"  Entraînables: {trainable_params:,}")
    print(f"  Non-entraînables: {non_trainable_params:,}")
    
except Exception as e:
    print(f"❌ Erreur lors de la construction du modèle: {e}")
    print("💡 Vérifiez la configuration du modèle")


In [ ]:
# Étape 3: Entraînement du modèle
print("🎯 Étape 3/3: Entraînement du modèle...")
print(f"⏰ Début de l'entraînement: {datetime.now().strftime('%H:%M:%S')}")

try:
    # Lancer l'entraînement
    trainer.train()
    
    print("🎉 Entraînement terminé avec succès!")
    print(f"⏰ Fin de l'entraînement: {datetime.now().strftime('%H:%M:%S')}")
    print(f"💾 Modèle sauvegardé dans: {trainer.output_dir}")
    
    # Afficher les métriques finales si disponibles
    if trainer.history:
        final_epoch = len(trainer.history.history['accuracy']) - 1
        final_acc = trainer.history.history['val_accuracy'][final_epoch]
        final_loss = trainer.history.history['val_loss'][final_epoch]
        
        print(f"\n📊 Résultats finaux:")
        print(f"  Précision validation: {final_acc:.4f}")
        print(f"  Perte validation: {final_loss:.4f}")
        
except Exception as e:
    print(f"❌ Erreur lors de l'entraînement: {e}")
    print("💡 Vérifiez les logs ci-dessus pour plus de détails")


In [ ]:
# Visualisation des résultats d'entraînement
if hasattr(trainer, 'history') and trainer.history:
    history = trainer.history.history
    
    # Créer les graphiques
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Précision
    axes[0, 0].plot(history['accuracy'], label='Entraînement', linewidth=2)
    axes[0, 0].plot(history['val_accuracy'], label='Validation', linewidth=2)
    axes[0, 0].set_title('Précision du Modèle', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Époque')
    axes[0, 0].set_ylabel('Précision')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Perte
    axes[0, 1].plot(history['loss'], label='Entraînement', linewidth=2)
    axes[0, 1].plot(history['val_loss'], label='Validation', linewidth=2)
    axes[0, 1].set_title('Perte du Modèle', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Époque')
    axes[0, 1].set_ylabel('Perte')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Top-3 Accuracy
    if 'top_3_accuracy' in history:
        axes[1, 0].plot(history['top_3_accuracy'], label='Entraînement Top-3', linewidth=2)
        axes[1, 0].plot(history['val_top_3_accuracy'], label='Validation Top-3', linewidth=2)
        axes[1, 0].set_title('Précision Top-3', fontsize=14, fontweight='bold')
        axes[1, 0].set_xlabel('Époque')
        axes[1, 0].set_ylabel('Précision Top-3')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
    
    # Top-5 Accuracy
    if 'top_5_accuracy' in history:
        axes[1, 1].plot(history['top_5_accuracy'], label='Entraînement Top-5', linewidth=2)
        axes[1, 1].plot(history['val_top_5_accuracy'], label='Validation Top-5', linewidth=2)
        axes[1, 1].set_title('Précision Top-5', fontsize=14, fontweight='bold')
        axes[1, 1].set_xlabel('Époque')
        axes[1, 1].set_ylabel('Précision Top-5')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Résumé des performances
    print("📊 Résumé des Performances:")
    print(f"  Meilleure précision validation: {max(history['val_accuracy']):.4f}")
    print(f"  Perte finale validation: {history['val_loss'][-1]:.4f}")
    if 'val_top_3_accuracy' in history:
        print(f"  Meilleure précision Top-3: {max(history['val_top_3_accuracy']):.4f}")
    if 'val_top_5_accuracy' in history:
        print(f"  Meilleure précision Top-5: {max(history['val_top_5_accuracy']):.4f}")
        
else:
    print("⚠️ Aucun historique d'entraînement disponible pour la visualisation")
